In [1]:
R.Version()

$platform
[1] "x86_64-w64-mingw32"

$arch
[1] "x86_64"

$os
[1] "mingw32"

$system
[1] "x86_64, mingw32"

$status
[1] ""

$major
[1] "4"

$minor
[1] "1.3"

$year
[1] "2022"

$month
[1] "03"

$day
[1] "10"

$`svn rev`
[1] "81868"

$language
[1] "R"

$version.string
[1] "R version 4.1.3 (2022-03-10)"

$nickname
[1] "One Push-Up"

In [ ]:
 Sys.setenv(JAGS_HOME = "C:/Program Files/JAGS/JAGS-4.0.0")

In [ ]:
.libPaths("C:/Users/jkmer/anaconda3/envs/r4-base/Library")

load.lib <- c("lme4",
              "ggplot2",
              "ggpubr",
              "directlabels",
              "RColorBrewer",
              "ggeffects",
              "see",
              "ggforce",
              "yarrr",
              "mgcv",
              "gratia",
              "tidymv",
              "visreg",
              "tidyverse",
              "dplyr",
              "directlabels",
              "confintr",
              "Kendall",
              "poolr",
              "broom",
              "modelr",
              "gdata",
             "LongituRF",
             "janitor",
             "zoo",
             "gt",
             "scales",
             "glue",
             "purrr",
             "htree",
             "JMbayes",
             "car",
             "agricolae",
             "flextable",
             "crul",
             "rempsyc"
) 

sapply(load.lib,require,character=TRUE)

In [ ]:
# read in cleaned CLinical Severity Scores from Rett Natural History Study 5201 and 5211
clean <- read.csv('C:/Users/jkmer/Desktop/2023_css/20230201_CSS_5201_5211_cleaned.csv')

In [ ]:
#convert id's etc to factors
clean$participant_id <- as.factor(clean$participant_id)
clean$childs_gender <- as.factor(clean$childs_gender)
clean$diagnosis <- as.factor(clean$diagnosis)
clean$grouping3 <- as.factor(clean$grouping3)
clean$visit <- as.factor(clean$visit)

In [ ]:
#round age at visit to 0.1 years
clean$age <- round(clean$age_at_visit, 1)

In [ ]:
#sort by age at visit within participant_id
 clean <- clean[
      with(clean, order(participant_id, age_at_visit)),]
        

In [ ]:
#filter for females with a mecp2 mutation
fem <- clean %>% filter(childs_gender == "Female")
fempos <- fem %>% filter(grouping1_genetic_mutation == "MECP2 mutation")

In [ ]:
#plot of females in rnhs with a mecp2 mutation
testpal <- c('#ff00ff', '#33cc33', '#006600',
             '#33cccc', '#777b7e', '#663300',
             '#cc9900', '#ff5050', '#3366ff',
             '#6600cc', '#ff9900')

My_Theme = theme(
  axis.title.x = element_text(size = 30, face="bold", color = "black"),
  axis.text.x = element_text(size = 30, face="bold", color = "black"),
  axis.title.y = element_text(size = 30, face="bold", color = "black"),
  axis.text.y = element_text(size = 30, face="bold", color = "black"),  
  axis.line = element_line(size = 2, color = "black"),
  axis.ticks = element_line(size = 2, color = "black"),
  legend.text = element_text(size = 30, face="bold", color = "black"),
  axis.ticks.length=unit(0.1,"inch"),    
  legend.title = element_blank())

baseline <- fempos %>% group_by(participant_id) %>% slice(which.min(age_at_visit)) %>%
                             ungroup()
dim(baseline)

rnhs_summary_plot <- baseline %>%     
    ggplot() +
    geom_jitter(aes(x=age, y=total_score_clean), size = 3, width = 0.01, height = 0.5) +
    scale_colour_manual(values = testpal) +
    theme_classic() +
    My_Theme +
    ylab('Baseline CSS') +
    xlab('Age (Years)') +
    geom_smooth(data = fempos,
                aes(x = age, y = total_score_clean), 
                formula = y ~ log(x), size = 4, se = FALSE, color = '#ff5050') +
    xlim(0,25)

png(file="C:/Users/jkmer/Desktop/2023_css/rnhs_summary.png",
units="in", width=8, height=8, res=300)
rnhs_summary_plot
dev.off()

rnhs_summary_plot

In [ ]:
#plot common mutations to show differences in mean baseline severity
stderror <- function(x) sd(x)/sqrt(length(x))
baseline_mean <- baseline %>% group_by(grouping3) %>% mutate(mean = mean(total_score_clean)) %>%
                         mutate(sem = stderror(total_score_clean)) %>% ungroup() %>%
                         select(c(grouping3, mean, sem)) %>% unique()
                         


My_Theme = theme(
  axis.title.x = element_blank(), #element_text(size = 20, face="bold", color = "black"),
  axis.text.x = element_text(size = 30, face="bold", color = "black", angle = 45, vjust = 1, hjust=1),
  axis.title.y = element_text(size = 30, face="bold", color = "black"),
  axis.text.y = element_text(size = 30, face="bold", color = "black"),  
  axis.line=element_line(size = 2, color = "black"),
  axis.ticks = element_line(size = 2, color = "black"),
  axis.ticks.length=unit(0.1,"inch"),    
  legend.position = "none")

baseline_mean$grouping3 <- factor(baseline_mean$grouping3, levels = c("earlytrunc",
                                 "R106W",
                                 "R133C",
                                 "T158M",
                                 "R168X",
                                 "R255X",
                                 "R270X",
                                 "R294X",
                                 "R306C",
                                 "LargeDel",
                                 "CTT")) 
baseline_mean <-  baseline_mean %>% na.omit()

rnhs_meanplot <- ggplot(baseline_mean, aes(reorder(grouping3,mean), mean, fill = grouping3)) + 
      geom_bar(stat = "identity", color = "black", size=1, width=0.9) +
      #stat_summary(fun=mean, geom="bar") +
      geom_errorbar(aes(ymin=mean-sem, ymax=mean+sem), width=.3, size=1) +
      scale_y_continuous(expand = c(0,0),
                     limits = c(0,37)) +
          ylab("Baseline CSS") +
          theme_classic() +
          scale_fill_manual(values = testpal) +
          My_Theme

png(file="C:/Users/jkmer/Desktop/2023_css/rnhs_baseline_summary.png",
units="in", width=8, height=8, res=300)
rnhs_meanplot
dev.off()

rnhs_meanplot

In [ ]:
#pairwise comparison between genotype baseline scores
#heatplot to show comparisons in mean prediction
baseline <- baseline %>% filter(grouping3 != "Splice") %>%
                         filter(grouping3 != "Exon1") %>%
                         filter(grouping3 != "OtherPt") 

comparison <- HSD.test(lm(total_score_clean~grouping3, data = baseline), "grouping3", group=FALSE)
pvals <- comparison$comparison %>% select(pvalue)
pvals$comp <- rownames(pvals)
pvals <- pvals %>%  mutate(signif = case_when(pvalue < 0.001 ~ '< 0.001',
  pvalue < 0.01 ~ '< 0.01',
  pvalue < 0.05 ~ '< 0.05',
  pvalue > 0.05 ~ '1'))
pvals <- separate(data = pvals, col = comp, into = c("A", "B"), sep = " - ")
pvals2 <- pvals
colnames(pvals2) <- c('pvalue', 'B', 'A', "signif")
pvals <- rbind(pvals, pvals2)
pvals

#plot pairwise comparisons
My_Theme = theme(
  axis.title.x = element_blank(), #element_text(size = 20, face="bold", color = "black"),
  axis.text.x = element_text(size = 30, face = "bold", color = "black", angle = 90, vjust = 0.5, hjust=1),
  axis.title.y = element_blank(),
  axis.text.y = element_text(size = 30, face = "bold", color = "black"),  
  axis.line=element_line(size = 2, color = "black"),
  axis.ticks = element_line(size = 2, color = "black"),
  axis.ticks.length=unit(0.1,"inch"),    
  legend.text = element_text(size=30, face = "bold", color = "black"),
  legend.title = element_text(size = 30, face = "bold", color = "black"))

level_order <- c('R133C',
                 'CTT',
                 'R306C',
                 'R294X',
                 'R106W',
                 'T158M',
                 'earlytrunc', 
                 'LargeDel',               
                 'R255X',
                 'R270X',
                 'R168X')



pvalplot <- pvals %>%
 ggplot(aes(x = factor(A, level = level_order), y = factor(B, level = level_order), fill = signif)) +
 geom_tile(color = "black") +
theme_classic() +
My_Theme +
 scale_x_discrete(expand=c(0.02,0)) +
 scale_y_discrete(expand=c(0.02,0)) +
scale_fill_manual(values = c("#FF6666", "#FFB266", "#66FFFF"),
                              name = "Tukey's p", 
                              labels = c('<0.001', '<0.01', 'NS')) +
coord_equal() +
theme(panel.border=element_rect(fill = NA, colour='black', size=2)) +
annotate("rect", xmin=c(0.5, 4.5), 
                 xmax=c(4.5, 11.5), 
                 ymin=c(0.5, 4.5), 
                 ymax=c(4.5, 11.5), colour="black", fill="transparent", size=2)

png(file="C:/Users/jkmer/Desktop/2023_css/pvalues_baselineCSS_allmut.png",
units="in", width=9, height=9, res=300)
pvalplot
dev.off()

pvalplot





In [ ]:
fempos$age <- round(fempos$age_at_visit, digits = 1)
fempos25 <- fempos %>% filter(age <= 25)

In [ ]:
femposatypical <- fempos25 %>% filter(diagnosis != "Classic")

In [ ]:
femposatypical <- femposatypical %>% select(c(participant_id, grouping3)) %>% unique()

In [ ]:
table(femposatypical$grouping3)

In [ ]:
femposclassic <- fempos %>% filter(diagnosis == "Classic")
length(unique(femposclassic$participant_id))

In [ ]:
femposclassic$age <- round(femposclassic$age_at_visit, digits = 1)
age25 <- femposclassic %>% filter(age <= 25)
length(unique(age25$participant_id))

In [ ]:
mutspec <- age25 %>% filter(grouping3 == "earlytrunc")
length(unique(mutspec$participant_id))

In [ ]:
#for model development, focus on individuals with a classic diagnosis
#filter for classic cases in females under 25 years old
classic <- clean[clean$diagnosis == "Classic" ,]
females <- classic[classic$childs_gender == "Female" ,]
mecp2_pos <- females[females$grouping1_genetic_mutation == "MECP2 mutation" ,]
mecp2_pos$age <- round(mecp2_pos$age_at_visit, digits = 1)
mecp2_pos$lnage <- log(mecp2_pos$age)
mecp2_pos25 <- mecp2_pos[mecp2_pos$age<=25,]

In [ ]:
#remove under-represented mutation groups
mecp2_pos25 <- mecp2_pos25 %>% filter(!grouping3 == 'OtherPt') %>% 
                               filter(!grouping3 == 'MISSING') %>% 
                               filter(!grouping3 == "Splice") %>%  
                               filter(!grouping3 == "Exon1")

mecp2_pos25 <- mecp2_pos25[!(is.na(mecp2_pos25$grouping3)), ]

In [ ]:
length(unique(mecp2_pos25$participant_id))

In [ ]:
dim(mecp2_pos25)

In [ ]:
5469/1003

In [ ]:
unique(mecp2_pos25$grouping3)

## disaggregating within and between person effects by decomposing time in the log transformed model

In [ ]:
#decompose logtime
mecp2_decomp <- mecp2_pos25 %>%
  group_by(participant_id) %>%
  summarize(mean_lnage = mean(lnage)) %>% 
  ungroup() %>%
  mutate(mean_lnage_ctr = scale(mean_lnage, scale = F)[,1],
         ctrval = attr(scale(mean_lnage, scale = F),"scaled:center")) %>% 
  right_join(mecp2_pos25, by = "participant_id") %>%
  mutate(duration = lnage - mean_lnage) %>% 
  select(participant_id, age, lnage, 
         mean_lnage, ctrval, mean_lnage_ctr, duration, 
         total_score_clean, grouping3)

In [ ]:
#logtime decomp
log_decomp <- lme(fixed = total_score_clean ~ mean_lnage_ctr*duration*grouping3, 
             random = ~duration|participant_id,
             data = mecp2_decomp,
             correlation = corAR1(form = ~1|participant_id),
             method = "ML")

In [ ]:
summary(log_decomp)

In [ ]:
#prediction of fixed effects
time <- seq(2,25, by = 0.1)
fixed_pred <- mecp2_pos25 %>% ungroup() %>% select(grouping3) %>% unique()
fixed_pred <- fixed_pred %>% group_by(grouping3) %>% tidyr::expand(age = time) %>% ungroup
fixed_pred$lnage <- log(fixed_pred$age)
fixed_pred$mean_lnage <- mean(log(time))
fixed_pred$mean_lnage_ctr <- 0
fixed_pred$duration <- fixed_pred$lnage - fixed_pred$mean_lnage

In [ ]:
fixed_pred$pred <- predict(log_decomp, fixed_pred, level=0)

In [ ]:
#logtime decomp model without inclusion of genotype for general modeling 
log_decomp_general <- lme(fixed = total_score_clean ~ mean_lnage_ctr*duration, 
             random = ~duration|participant_id,
             data = mecp2_decomp,
             correlation = corAR1(form = ~1|participant_id),
             method = "ML")

In [ ]:
#prediction of fixed effects
time <- seq(2,25, by = 0.1)
fixed_pred_general <- as.data.frame(time)
colnames(fixed_pred_general) <- c('age')
fixed_pred_general$lnage <- log(fixed_pred_general$age)
fixed_pred_general$mean_lnage <- mean(log(time))
fixed_pred_general$mean_lnage_ctr <- 0
fixed_pred_general$duration <- fixed_pred_general$lnage - fixed_pred_general$mean_lnage

In [ ]:
fixed_pred_general$pred <- predict(log_decomp_general, fixed_pred_general, level=0)

In [ ]:
#col_pal <- unname(piratepal('xmen', length.out = 8))
#col_pal <-c("#999999", "#E69F00", "#56B4E9", "#009E73", 
 #          "#F0E442", "#0072B2", "#D55E00", "#CC79A7")
#cbp1 <- c("#999999", "#E69F00", "#56B4E9", "#009E73",
 #         "#F0E442", "#0072B2", "#D55E00", "#CC79A7")


My_Theme = theme(
  axis.title.x = element_text(size = 30, face="bold", color = "black"),
  axis.text.x = element_text(size = 30, face="bold", color = "black"),
  axis.title.y = element_text(size = 30, face="bold", color = "black"),
  axis.text.y = element_text(size = 30, face="bold", color = "black"),  
  axis.line = element_line(size = 2, color = "black"),
  axis.ticks = element_line(size = 2, color = "black"),
  legend.text = element_text(size = 30, face="bold", color = "black"),
  axis.ticks.length=unit(0.1,"inch"),    
  legend.title = element_blank())

fixed_pred$grouping3 <-  factor(fixed_pred$grouping3, levels = c("earlytrunc",
                                 "R106W",
                                 "R133C",
                                 "T158M",
                                 "R168X",
                                 "R255X",
                                 "R270X",
                                 "R294X",
                                 "R306C",
                                 "LargeDel",
                                 "CTT")) 

logdecompplot <-  fixed_pred %>%
    ggplot() +
    geom_smooth(aes(x=age, y=pred, color = grouping3), size = 4, se = FALSE) +
    scale_colour_manual(values = testpal) +
    theme_classic() +
    My_Theme +
    guides(color=guide_legend(ncol=4)) +
    ylab('CSS') +
    xlab('Age (Years)') +
            theme(legend.position = c(0.5, 0.95)) +
    geom_smooth(data = fixed_pred_general,
                aes(x = age, y = pred), size = 4, se = FALSE, color = 'black', linetype = 11 ) +
    ylim(0,32)

png(file="C:/Users/jkmer/Desktop/2023_css/logdecompmodel_allmut.png",
units="in", width=10, height=16, res=300)
logdecompplot
dev.off()

logdecompplot

## Individual predictions from log decomp model using bayesian approach

In [ ]:
#pull center value from training data set
ctr_val_train <- unique(mecp2_decomp$ctrval)

In [ ]:
#predict decomposed model for all individuals with represented mutations in mecp2 regardless of diagnosis


#filter parent data for females with represented mecp2 mutations
females <- clean[clean$childs_gender == "Female" ,]
mecp2_pos <- females[females$grouping1_genetic_mutation == "MECP2 mutation" ,]
mecp2_pos$age <- round(mecp2_pos$age_at_visit, digits = 1)
mecp2_pos$lnage <- log(mecp2_pos$age)
mecp2_pos <- mecp2_pos %>% filter( age <= 25)
mecp2_pos$participant_id <- factor(mecp2_pos$participant_id)

#remove under represented mutation groupings
mecp2_pos <- mecp2_pos %>% filter(!grouping3 == 'OtherPt') %>% 
                               filter(!grouping3 == 'MISSING') %>% 
                               filter(!grouping3 == "Splice") %>%  
                               filter(!grouping3 == "Exon1")
mecp2_pos <- mecp2_pos[!(is.na(mecp2_pos$grouping3)), ]

#decompose age
mecp2_pos_decomp <- mecp2_pos %>%
  group_by(participant_id) %>%
  summarize(mean_lnage = mean(lnage)) %>% # calculates the mean age per person
  ungroup() %>%
mutate(mean_lnage_ctr = mean_lnage - ctr_val_train) %>% 
  right_join(mecp2_pos, by = "participant_id") %>% 
  mutate(duration = lnage - mean_lnage) %>%
  select(participant_id, age, lnage, 
         mean_lnage, mean_lnage_ctr, duration, 
         total_score_clean, grouping3)

#assign cntrval from training
mecp2_pos_decomp$ctrval <- ctr_val_train

#assign numeric factors to grouping3 in training and prediction - necessary for running JMbayes
mecp2_decomp$grp_3num <- as.factor(as.numeric(mecp2_decomp$grouping3))
mecp2_pos_decomp$grp_3num <- as.factor(as.numeric(mecp2_pos_decomp$grouping3))

#create vector of ages for prediction
time <- seq(2, 25, by = 0.1) #by = 1 for low resolution testing, by = 0.1 for actual production run
logtime <- log(time)
       
#create vector of participant_id's for predictions to be made on       
participant_ids <- unique(mecp2_pos_decomp$participant_id) #ref all diagnosis part_ids

#train log decomp model on classic diagnosis       
log_decomp <- lme(fixed = total_score_clean ~ mean_lnage_ctr*duration*grp_3num, 
             random = ~duration|participant_id,
             data = mecp2_decomp,
             correlation = corAR1(form = ~1|participant_id),
             method = "ML")      
       
#loop over participant_ids for all diagnosis, constructing new set of durations based on individual specific mean_lnage
all_pred = data.frame()
for (i in participant_ids){
     
        #filter all diagnosis for participant_id of interest
        tmp_new_dat <- mecp2_pos_decomp %>% filter(participant_id == i)
    
        #create vector of individual specific durations based on mean_lnage
        tmp_durations <- logtime - unique(tmp_new_dat$mean_lnage)
        
    
        #predict with JMbayes        
        indiv_pred <- IndvPred_lme(log_decomp, 
                           newdata = tmp_new_dat, 
                           timeVar = "duration", 
                           M = 500, 
                           return_data = FALSE, #set to false for forecasting, if export with dataframe, need to remove rows of original data                           seed = 321,
                           all_times = TRUE,
                           times = tmp_durations)

        #create data frame with individual and grouping3, assign ages of prediction, merge with predicted total_score
        pred <- tmp_new_dat %>% 
                select(c(participant_id,grouping3)) %>% unique() %>% 
                group_by(participant_id, grouping3) %>% expand(age = time) %>% ungroup()
        pred$pred <- indiv_pred$predicted_y
    
        #add individual predictions to output
        all_pred <- rbind(all_pred, pred)
    }

In [ ]:
#pull individuals with > 13 visits and select single individual for each mutation class
multi_visits <- mecp2_pos_decomp %>% group_by(participant_id) %>% 
        filter(age <= 25) %>%
        filter(n() > 13) %>% ungroup() %>%
        select(c(participant_id, grouping3)) %>% 
        unique()
sample_multi <- multi_visits %>% distinct(grouping3, .keep_all = TRUE)

In [ ]:
#plotting predicted versus actual for randomly selected individuals with > 13 visits

My_Theme = theme(
  axis.title.x = element_text(size = 30, face="bold", color = "black"),
  axis.text.x = element_text(size = 30, face="bold", color = "black"),
  axis.title.y = element_text(size = 30, face="bold", color = "black"),
  axis.text.y = element_text(size = 30, face="bold", color = "black"),  
  axis.line = element_line(size = 2),
  axis.ticks = element_line(size = 2),
  axis.ticks.length=unit(0.1,"inch"),    
  legend.text = element_text(size = 30, color = "black"),
  legend.title = element_blank())


checkid <- sample_multi$participant_id
predcheck <- all_pred %>% filter(participant_id %in% checkid)
realcheck <- mecp2_pos_decomp %>% filter(participant_id %in% checkid)


fitplot <- ggplot() +
geom_smooth(data = predcheck, aes(x = age, y = pred), show.legend = FALSE, size = 4 ) +
geom_point(data = realcheck, aes(x = age, y = total_score_clean), show.legend = FALSE, size = 4 ) +
theme_classic() +
My_Theme +
xlab("Age (years)") +
ylab("CSS") +
facet_wrap(~grouping3, nrow = 4, scales = "free") +
xlim(0,25) +
ylim(0,45) + 
theme(
  strip.background = element_blank(),
  strip.text.x = element_text(size = 30, face="bold", color = "black") #element_blank()
)

png(file="C:/Users/jkmer/Desktop/2023_css/fit2.png",
units="in", width=20, height=20, res=300)
fitplot
dev.off()

fitplot




## RMSE LOOE for model performance

In [ ]:
#LOOE for model performance,
#Model input should be classic diagnosis, predictions are for all diagnosis
#make predictions for observed ages to save computational time
#calculate RMSE for all and by group
pred_input <- mecp2_decomp #set input for training model
participant_ids <- unique(mecp2_pos_decomp$participant_id) #reference diagnosis part_ids
rmse_pred = data.frame()

for (i in participant_ids){
        train <- subset(pred_input, participant_id != i) #train cannot include sample of interest
        test <- subset(mecp2_pos_decomp, participant_id == i) #pull single individual from all diagnosis
        time <- test$age
           if (length(time) == 1) {
                                time <- c(time,26)
                                } else {
                                        time <- time
                                                    }
    logtime <- log(time)
       
    #specify model   
    model_rep <- lme(fixed = total_score_clean ~ mean_lnage_ctr*duration*grp_3num, 
             random = ~duration|participant_id,
             data = train,
             correlation = corAR1(form = ~1|participant_id),
             method = "ML")  
        
    #create individual specific durations
        #filter all diagnosis for participant_id of interest
        tmp_new_dat <- mecp2_pos_decomp %>% filter(participant_id == i)
    
        #create vector of individual specific durations based on mean_lnage
        tmp_durations <- logtime - unique(tmp_new_dat$mean_lnage)
        
    
        tmp_pred <- IndvPred_lme(model_rep, 
                           newdata = test, 
                           timeVar = "duration", 
                           M = 500, 
                           return_data = TRUE,
                           seed = 321,
                           all_times = TRUE,
                           times = tmp_durations)
       
        rmse_pred <- rbind(rmse_pred, tmp_pred)
    
}

rmse_pred <- rmse_pred %>% filter(age != 26)

In [ ]:
write.csv(rmse_pred, "C:/Users/jkmer/Desktop/2023_css/rmse_pred.csv", row.names = FALSE)

In [ ]:
#calculate rmse
rmse <- rmse_pred %>% 
        group_by(participant_id) %>% 
        mutate(resid = pred - total_score_clean) %>%
        mutate(residsquare = resid^2) %>%
        mutate(mse = mean(residsquare)) %>%
        mutate(rmse = sqrt(mse))

In [ ]:
#plot rmse

My_Theme = theme(
  axis.title.x = element_blank(), #element_text(size = 20, face="bold", color = "black"),
  axis.text.x = element_text(size = 30, face="bold", color = "black", angle = 45, vjust = 1, hjust=1),
  axis.title.y = element_text(size = 30, face="bold", color = "black"),
  axis.text.y = element_text(size = 30, face="bold", color = "black"),  
  axis.line = element_line(size = 2, color = "black"),
  axis.ticks = element_line(size = 2, color = "black"),
  axis.ticks.length=unit(0.1,"inch"),    
  legend.position = "none")


rmse$grouping3 <- factor(rmse$grouping3, levels = c("earlytrunc",
                                 "R106W",
                                 "R133C",
                                 "T158M",
                                 "R168X",
                                 "R255X",
                                 "R270X",
                                 "R294X",
                                 "R306C",
                                 "LargeDel",
                                 "CTT"))  
level_order <- c("earlytrunc",
                                 "R106W",
                                 "R133C",
                                 "T158M",
                                 "R168X",
                                 "R255X",
                                 "R270X",
                                 "R294X",
                                 "R306C",
                                 "LargeDel",
                                 "CTT") 

rmseplot <- rmse %>% select(c(participant_id, grouping3, rmse)) %>%
         unique() %>%
         #filter(grouping3 != "LargeDel" & grouping3 != "earlytrunc" & grouping3 != "CTT") %>% 
         ggplot(aes(x=factor(grouping3, level = level_order), y=rmse, fill = grouping3)) + 
         stat_boxplot(geom ='errorbar', size = 2, color = "black") +
         geom_boxplot(size = 2, color = "black") +
         scale_fill_manual(values = testpal) +
         theme_classic() +
         My_Theme +
         ylab("RMSE")

png(file="C:/Users/jkmer/Desktop/2023_css/rmse2_allmut.png",
units="in", width=7, height=6, res=300)
rmseplot
dev.off()

rmseplot

## Calculate percentile distributions for mean pred score 
## for common cases, assign percentiles to all cases

In [ ]:
#percentile calculation with mean predicted by individual
#pull mean predictions and separate common cases
all_mean_pred <- all_pred %>% unique()
all_mean_pred <- all_mean_pred %>% group_by(participant_id) %>% mutate(mean_pred = mean(pred)) 
all_mean_pred <- all_mean_pred %>% select(c(participant_id, grouping3, mean_pred)) %>% unique()
classic_mean_pred <- all_mean_pred %>% filter(participant_id %in% mecp2_decomp$participant_id)

In [ ]:
write.csv(all_mean_pred, "C:/Users/jkmer/Desktop/2023_css/mean_pred.csv", row.names = FALSE)

In [ ]:
#calculate and assign percentiles to predicted terminal values, based on cumulative distributions 
#of terminal values
#create dummy array (TotalScores) with all grouping3 and score combinations at age 25

Grouping3 <- c("R106W", 
               "R133C", 
               "T158M", 
               "R168X", 
               "R255X", 
               "R270X", 
               "R294X", 
               "R306C",
               "CTT",
               "LargeDel",
               "earlytrunc")
fitted <- seq(from = 1, to = 58, by = 1)

TotalScores = data.frame(fitted)

TotalScores_mut <- TotalScores %>% 
    group_by(fitted) %>% 
    tidyr::expand(Grouping3)

TotalScores_df <- as.data.frame(TotalScores_mut)


In [ ]:
#for loop to calculate dummy array TotalScores_df percentiles 
#from cumulative dist. of predicted terminal values for common cases

scores_percentile = data.frame()

predictions <- classic_mean_pred

for (j in Grouping3) {
        pred_scores <- predictions[predictions$grouping3 == j ,]
        predict_fx <- ecdf(pred_scores$mean_pred)
        scores_restrict <- TotalScores_df[TotalScores_df$Grouping3 == j ,]
        scores_restrict$Percentile <- predict_fx(v = scores_restrict$fitted)
        rownames(scores_restrict) <- NULL
        # Using rbind() to append the output of one iteration to the dataframe
        scores_percentile = rbind(scores_percentile, scores_restrict)
        }
    
colnames(scores_percentile) = c( 'mean_pred', 'grouping3', 'Percentile')

In [ ]:
#merge percentiles with predicted terminal scores
#round predicted values and percentiles to whole
scores_percentile$Percentile <- round(scores_percentile$Percentile, 2)
all_mean_pred$mean_pred <- round(all_mean_pred$mean_pred)
all_mean_pred$grouping3 <- as.factor(all_mean_pred$grouping3)
#assign percentiles to estimated terminal scores for all diagnosis
individual_percentiles <- merge(all_mean_pred, scores_percentile, by=c("mean_pred","grouping3")) %>%
                           unique()
#pull classic diagnosis percentiles
classic_percentiles <- individual_percentiles %>% filter(participant_id %in% mecp2_decomp$participant_id)

In [ ]:
#export individual percentiles and all_mean_pred
write.csv(individual_percentiles, 
          "C:/Users/jkmer/Desktop/2023_css/percentiles.csv", row.names = FALSE)

In [ ]:
classic_percentiles <- individual_percentiles %>% filter(participant_id %in% mecp2_decomp$participant_id)

In [ ]:
#plot cdf percentiles calculated from classic
My_Theme = theme(
  axis.title.x = element_text(size = 30, face="bold", color = "black"),
  axis.text.x = element_text(size = 30, face="bold", color = "black"),
  axis.title.y = element_text(size = 30, face="bold", color = "black"),
  axis.text.y = element_text(size = 30, face="bold", color = "black"),  
  axis.line = element_line(size = 2, color = "black"),
  axis.ticks = element_line(size = 2, color = "black"),
  legend.text = element_text(size = 30, face = "bold", color = "black"),
  axis.ticks.length=unit(0.1,"inch"),    
  legend.title = element_blank())


individual_percentiles$grouping3 <- factor(individual_percentiles$grouping3, levels = c("earlytrunc",
                                 "R106W",
                                 "R133C",
                                 "T158M",
                                 "R168X",
                                 "R255X",
                                 "R270X",
                                 "R294X",
                                 "R306C",
                                 "LargeDel",
                                 "CTT")) 


distplot <- individual_percentiles %>% 
        ggplot() +
            geom_line(aes(x = mean_pred, y = 100*Percentile, color = grouping3), size = 3) +
        labs(color='Mutation') + 
        ylab("Percentile") + 
        xlab("Mean Predicted CSS") +
        theme_classic() + 
        scale_color_manual(values = testpal) +
        My_Theme + 
        theme(legend.position = c(0.85, 0.4)) +
                scale_y_continuous(expand = c(0,0),
                     limits = c(0,110)) +
        xlim(8,42)

png(file="C:/Users/jkmer/Desktop/2023_css/distributions_allmut.png",
units="in", width=16, height=8, res=300)
distplot
dev.off()

distplot

## Anova for assessing separation of genotypes by predicted terminal score

In [ ]:
#anova to check for grouping3 genotype separation in terminal scores "pred" in classic cases
#since groups are unbalanced, need to use Type III sums of squares
pred_aov <- Anova(lm(mean_pred~grouping3, data = classic_percentiles), type="III")
pred_aov

In [ ]:
comparison <- HSD.test(lm(mean_pred~grouping3, data = classic_percentiles), "grouping3", group=TRUE)
comparison

In [ ]:
#calculate mean predictons standard error by genotype group
stderror <- function(x) sd(x)/sqrt(length(x))
mean_pred2 <- classic_percentiles %>% group_by(grouping3) %>% 
                                    mutate(mean = mean(mean_pred)) %>%
                                    mutate(sem = stderror(mean_pred)) %>%
                                    select(grouping3, mean, sem) %>%
                                    unique()

In [ ]:
#reassign grouping letters in reverse for ease of presentation
groups <- comparison$groups
groups$grouping3 <- rownames(groups)
groups <- groups %>% select(c(groups, grouping3))
groups <- merge(groups, mean_pred2, by = "grouping3")
#groups$groups2 <- c('ab', 'bcd', 'bcd', 'bcd', 'a', 'cd', 'cd', 'd', 'a', 'a', 'bc')

In [ ]:
#filters <- c('earlytrunc', 'LargeDel', 'CTT')
#groups <- groups %>% filter(!grouping3 %in% filters)
groups$groups2 <- c('a', 'bc', 'bc', 'bc', 'a', 'c', 'c', 'c', 'a', 'a', 'b')

In [ ]:
groups

In [ ]:
#plot mean predictions

My_Theme = theme(
  axis.title.x = element_blank(), #element_text(size = 20, face="bold", color = "black"),
  axis.text.x = element_text(size = 30, face="bold", color = "black", angle = 45, vjust = 1, hjust=1),
  axis.title.y = element_text(size = 30, face="bold", color = "black"),
  axis.text.y = element_text(size = 30, face="bold", color = "black"),  
  axis.line=element_line(size = 2, color = "black"),
  axis.ticks = element_line(size = 2, color = "black"),
  axis.ticks.length=unit(0.1,"inch"),    
  legend.position = "none")

groups$grouping3 <- factor(groups$grouping3, levels = c("earlytrunc",
                                 "R106W",
                                 "R133C",
                                 "T158M",
                                 "R168X",
                                 "R255X",
                                 "R270X",
                                 "R294X",
                                 "R306C",
                                 "LargeDel",
                                 "CTT")) 

meanplot <- ggplot(groups, aes(reorder(grouping3,mean), mean, fill = grouping3)) + 
      geom_bar(stat = "identity", color = "black", size=1, width=0.9) +
      #stat_summary(fun=mean, geom="bar") +
      geom_errorbar(aes(ymin=mean-sem, ymax=mean+sem), width=.3, size=1) +
      scale_y_continuous(expand = c(0,0),
                     limits = c(0,37)) +
          ylab("Mean Pred. CSS") +
          theme_classic() +
          scale_fill_manual(values = testpal) +
          My_Theme +
        geom_text(aes(label = groups2, fontface = "bold"), vjust = -2, size = 7)


png(file="C:/Users/jkmer/Desktop/2023_css/mean_pred_allmut.png",
units="in", width=8, height=8, res=300)
meanplot
dev.off()

meanplot

In [ ]:
#heatplot to show comparisons in mean prediction
comparison <- HSD.test(lm(mean_pred~grouping3, data = classic_percentiles), "grouping3", group=FALSE)
pvals <- comparison$comparison %>% select(pvalue)

In [ ]:
pvals$comp <- rownames(pvals)
pvals <- pvals %>%  mutate(signif = case_when(pvalue < 0.001 ~ '< 0.001',
  pvalue < 0.01 ~ '< 0.01',
  pvalue < 0.05 ~ '< 0.05',
  pvalue > 0.05 ~ '1'))
pvals <- separate(data = pvals, col = comp, into = c("A", "B"), sep = " - ")
pvals2 <- pvals
colnames(pvals2) <- c('pvalue', 'B', 'A', "signif")
pvals <- rbind(pvals, pvals2)


In [ ]:
pvals

In [ ]:
#plot pairwise comparisons
My_Theme = theme(
  axis.title.x = element_blank(), #element_text(size = 20, face="bold", color = "black"),
  axis.text.x = element_text(size = 30, face = "bold", color = "black", angle = 90, vjust = 0.5, hjust=1),
  axis.title.y = element_blank(),
  axis.text.y = element_text(size = 30, face = "bold", color = "black"),  
  axis.line=element_line(size = 2, color = "black"),
  axis.ticks = element_line(size = 2, color = "black"),
  axis.ticks.length=unit(0.1,"inch"),    
  legend.text = element_text(size=30, face = "bold", color = "black"),
  legend.title = element_text(size = 30, face = "bold", color = "black"))

level_order <- c('R306C',
                 'R133C',
                 'R294X',
                 'CTT',
                 'T158M',
                 'LargeDel',
                 'R106W',
                 'earlytrunc',                 
                 'R168X',
                 'R255X',
                 'R270X')



pvalplot <- pvals %>%
 ggplot(aes(x = factor(A, level = level_order), y = factor(B, level = level_order), fill = signif)) +
 geom_tile(color = "black") +
theme_classic() +
My_Theme +
 scale_x_discrete(expand=c(0.02,0)) +
 scale_y_discrete(expand=c(0.02,0)) +
scale_fill_manual(values = c("#FF6666", "#FFB266", "#FFFF66", "#66FFFF"),
                              name = "Tukey's p", 
                              labels = c('<0.001', '<0.01', '<0.05', 'NS')) +
coord_equal() +
theme(panel.border=element_rect(fill = NA, colour='black', size=2)) +
annotate("rect", xmin=c(0.5, 4.5), 
                 xmax=c(4.5, 11.5), 
                 ymin=c(0.5, 4.5), 
                 ymax=c(4.5, 11.5), colour="black", fill="transparent", size=2)

png(file="C:/Users/jkmer/Desktop/2023_css/pvalues_allmut.png",
units="in", width=9, height=9, res=300)
pvalplot
dev.off()

pvalplot


## Distribution of percentiles to show collapsing of genotype - phenotype in the nCSS score

In [ ]:
#calculate and assign percentiles to predicted terminal values, based on cumulative distributions 
#of terminal values
#create dummy array (TotalScores) with all grouping3 and score combinations at age 25

Grouping3 <- c("R106W", 
               "R133C", 
               "T158M", 
               "R168X", 
               "R255X", 
               "R270X", 
               "R294X", 
               "R306C",
               "CTT",
               "LargeDel",
               "earlytrunc")
fitted <- seq(from = 0, to = 1, by = 0.01)

TotalScores = data.frame(fitted)

TotalScores_mut <- TotalScores %>% 
    group_by(fitted) %>% 
    tidyr::expand(Grouping3)

TotalScores_df <- as.data.frame(TotalScores_mut)


In [ ]:
#for loop to calculate dummy array TotalScores_df percentiles 
#from cumulative dist. of predicted terminal values for common cases

scores_percentile = data.frame()

predictions <- individual_percentiles

for (j in Grouping3) {
        pred_scores <- predictions[predictions$grouping3 == j ,]
        predict_fx <- ecdf(pred_scores$Percentile)
        scores_restrict <- TotalScores_df[TotalScores_df$Grouping3 == j ,]
        scores_restrict$Percentile <- predict_fx(v = scores_restrict$fitted)
        rownames(scores_restrict) <- NULL
        # Using rbind() to append the output of one iteration to the dataframe
        scores_percentile = rbind(scores_percentile, scores_restrict)
        }
    
colnames(scores_percentile) = c( 'Percentile', 'grouping3', 'Percentile2')

In [ ]:
#assign percentiles to estimated terminal scores for all diagnosis
individual_percentiles2 <- merge(individual_percentiles, scores_percentile, by=c("Percentile","grouping3")) %>%
                           unique()


In [ ]:
#plot cdf percentiles calculated from common percentiles
My_Theme = theme(
  axis.title.x = element_text(size = 30, face="bold", color = "black"),
  axis.text.x = element_text(size = 30, face="bold", color = "black"),
  axis.title.y = element_text(size = 30, face="bold", color = "black"),
  axis.text.y = element_text(size = 30, face="bold", color = "black"),  
  axis.line = element_line(size = 2, color = "black"),
  axis.ticks = element_line(size = 2, color = "black"),
  legend.text = element_text(size = 30, face = "bold", color = "black"),
  legend.title = element_blank())#element_text(size = 20, face="bold", color = "black"))


individual_percentiles2$grouping3 <- factor(individual_percentiles2$grouping3, levels = c("earlytrunc",
                                 "R106W",
                                 "R133C",
                                 "T158M",
                                 "R168X",
                                 "R255X",
                                 "R270X",
                                 "R294X",
                                 "R306C",
                                 "LargeDel",
                                 "CTT")) 


distplot <- individual_percentiles2 %>% 
        #filter(grouping3 != "LargeDel" & grouping3 != "earlytrunc" & grouping3 != "CTT") %>% 
        ggplot() +
            #geom_point(aes(x = pred, y = Percentile, color = grouping3)) +
            geom_line(aes(x = Percentile, y = 100*Percentile2, color = grouping3), size = 3) +
        labs(color='Mutation') + 
        ylab("Percentile") + 
        xlab("Normative CSS") +
        theme_classic() + 
        scale_color_manual(values = testpal) +
        My_Theme + 
        theme(legend.position = c(0.85, 0.4)) +
                scale_y_continuous(expand = c(0,0),
                     limits = c(0,110))# +
       # xlim(0,120)

png(file="C:/Users/jkmer/Desktop/2023_css/distributions_allmut_percentiles.png",
units="in", width=16, height=8, res=300)
distplot
dev.off()

distplot